In [1]:
import pandas as pd
import numpy as np
import torch
from torch import cuda
from torch.utils.data import Dataset, DataLoader
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
from load_data import initialize_data
from reading_datasets import read_task
from labels_to_ids import labels_to_ids_tam
import time
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
def train(epoch, train_loader, model, optimizer, device, grad_step = 1, max_grad_norm = 10):
    tr_loss, tr_accuracy = 0, 0
    tr_precision, tr_recall = 0, 0
    tr_f1score = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    optimizer.zero_grad()
    
    for idx, batch in enumerate(train_loader):
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        if (idx + 1) % 20 == 0:
            print('FINSIHED BATCH:', idx, 'of', len(train_loader))

        #loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
        output = model(input_ids=ids, attention_mask=mask, labels=labels)
        tr_loss += output[0]

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = output[1].view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
        
        # Compute Precision
        tmp_tr_precision = precision_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0 )
        tr_precision += tmp_tr_precision
        
        # Compute Recall
        tmp_tr_recall = recall_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0)
        tr_recall += tmp_tr_recall
        
        # Compute f1score
        tmp_tr_f1score = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(), average= 'macro', zero_division=0)
        tr_f1score += tmp_tr_f1score
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=max_grad_norm
        )
        
        # backward pass
        output['loss'].backward()
        if (idx + 1) % grad_step == 0:
            optimizer.step()
            optimizer.zero_grad()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    tr_precision = tr_precision / nb_tr_steps
    tr_recall = tr_recall / nb_tr_steps
    tr_f1score= tr_f1score / nb_tr_steps
    #print(f"Training loss epoch: {epoch_loss}")
    #print(f"Training accuracy epoch: {tr_accuracy}")

    return model

In [3]:
def testing(model, testing_loader, labels_to_ids, device):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    eval_precision, eval_recall = 0, 0
    eval_f1score = 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
     
    
    ids_to_labels = dict((v,k) for k,v in labels_to_ids.items())

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            #loss, eval_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
            output = model(input_ids=ids, attention_mask=mask, labels=labels)

            eval_loss += output['loss'].item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = output[1].view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy
            
            # Compute Precision
            tmp_eval_precision = precision_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0)
            eval_precision += tmp_eval_precision
            
            # Compute Recall
            tmp_eval_recall = recall_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0)
            eval_recall += tmp_eval_recall
            
            # Compute f1score
            tmp_eval_f1score = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(), average='macro', zero_division=0)
            eval_f1score += tmp_eval_f1score

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    eval_precision = eval_precision / nb_eval_steps
    eval_recall = eval_recall / nb_eval_steps
    eval_f1score = eval_f1score / nb_eval_steps
    #print(f"Validation Loss: {eval_loss}")
    #print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions, eval_accuracy, eval_precision, eval_recall, eval_f1score

In [4]:
def main(n_epochs, model_name, model_save_flag, model_save_location, model_load_flag, model_load_location):
    #Initialization training parameters
    max_len = 256
    batch_size = 32
    grad_step = 1
    learning_rate = 1e-05
    initialization_input = (max_len, batch_size)

    #Reading datasets and initializing data loaders
    dataset_location = '../datasets/task_a/'

    train_data = read_task(dataset_location , split = 'tam_sentiment_train')
    dev_data = read_task(dataset_location , split = 'tam_sentiment_dev')
    #test_data = read_task(dataset_location , split = 'dev')#load test set
    labels_to_ids = labels_to_ids_tam
    #input_data = (train_data, dev_data, labels_to_ids)

    #Define tokenizer, model and optimizer
    device = 'cuda' if cuda.is_available() else 'cpu' #save the processing time
    if model_load_flag:
        tokenizer = AutoTokenizer.from_pretrained(model_load_location)
        model = AutoModelForSequenceClassification.from_pretrained(model_load_location)
    else: 
        tokenizer =  AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(labels_to_ids))
    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
    model.to(device)

    #Get dataloaders
    train_loader = initialize_data(tokenizer, initialization_input, train_data, labels_to_ids, shuffle = True)
    dev_loader = initialize_data(tokenizer, initialization_input, dev_data, labels_to_ids, shuffle = True)
    #test_loader = initialize_data(tokenizer, initialization_input, test_data, labels_to_ids, shuffle = True)#create test loader

    best_dev_acc = 0
    best_test_acc = 0
    best_dev_precision = 0
    best_test_precision = 0
    best_dev_recall = 0
    best_test_recall = 0
    best_dev_f1score = 0
    best_test_f1score = 0
    best_epoch = -1
    
    list_dev_acc = [] 
    list_test_acc = []  
    list_dev_precision = []  
    list_test_precision  = []  
    list_dev_recall = []  
    list_test_recall = []  
    list_dev_f1score = []  
    list_test_f1score = []
    
    for epoch in range(n_epochs):
        start = time.time()
        print(f"Training epoch: {epoch + 1}")

        #train model
        if not model_load_flag:
            model = train(epoch, train_loader, model, optimizer, device, grad_step)
        
        #testing and logging
        labels_dev, predictions_dev, dev_accuracy, dev_precision, dev_recall, dev_f1score = testing(model, dev_loader, labels_to_ids, device)
        print('DEV ACC:', dev_accuracy)
        print('DEV Precision:' , dev_precision)
        print('DEV Recall:' , dev_recall)
        print('DEV F1Score:' , dev_f1score)
        
        list_dev_acc.append(dev_accuracy)     
        list_dev_precision.append(dev_precision)   
        list_dev_recall.append(dev_recall)  
        list_dev_f1score.append(dev_f1score)  
        
        
        #labels_test, predictions_test, test_accuracy, test_precision, test_recall, test_f1score = testing(model, test_loader, labels_to_ids, device)
        #print('TEST ACC:', test_accuracy)
        #print('TEST Precision:' , test_precision)
        #print('TEST Recall:' , test_recall)
        #print('TEST F1Score:' , test_f1score)
        
        #list_test_acc.append(test_accuracy) 
        #list_test_precision.append(test_precision)  
        #list_test_recall.append(test_recall)
        #list_test_f1score.append(test_f1score) 

        #saving model
        if dev_accuracy > best_dev_acc:
            best_dev_acc = dev_accuracy
            #best_test_acc = test_accuracy
        if dev_precision > best_dev_precision:
            best_dev_precision = dev_precision
            #best_test_precision = test_precision
        if dev_recall > best_dev_recall:
            best_dev_recall = dev_recall
            #best_test_recall = test_recall
        if dev_f1score > best_dev_f1score:
            best_dev_f1score = dev_f1score
            #best_test_f1score = test_f1score
            best_epoch = epoch
            
            if model_save_flag:
                os.makedirs(model_save_location, exist_ok=True)
                tokenizer.save_pretrained(model_save_location)
                model.save_pretrained(model_save_location)

        now = time.time()
        print('BEST ACCURACY --> ', 'DEV:', round(best_dev_acc, 5))
        print('BEST PRECISION --> ', 'DEV:', round(best_dev_precision, 5))
        print('BEST RECALL --> ', 'DEV:', round(best_dev_recall, 5))
        print('BEST F1SCORE --> ', 'DEV:', round(best_dev_f1score, 5))
        print('TIME PER EPOCH:', (now-start)/60 )
        print()

    return best_dev_acc, best_test_acc, best_epoch, best_dev_precision, best_test_precision, best_dev_recall, best_test_recall, best_dev_f1score, best_test_f1score, list_dev_acc, list_test_acc, list_dev_precision, list_test_precision, list_dev_recall, list_test_recall, list_dev_f1score, list_test_f1score

In [5]:
if __name__ == '__main__':
    n_epochs = 10
    models = ['bert-base-multilingual-uncased']
    
    #model saving parameters
    model_save_flag = True
    model_load_flag = False
    
    overall_list_dev_acc = [] 
    overall_list_test_acc = []    
    overall_list_dev_precision = []  
    overall_list_test_precision  = []  
    overall_list_dev_recall = []  
    overall_list_test_recall = []  
    overall_list_dev_f1score = []  
    overall_list_test_f1score = [] 
    
    for i in range(5):
        
        for model_name in models:

            model_save_location = 'saved_models/' + model_name + 'Tamil' + str(i)
            model_load_location = None

            best_dev_acc, best_test_acc, best_epoch, best_dev_precision, best_test_precision, best_dev_recall, best_test_recall, best_dev_f1score, best_test_f1score, list_dev_acc, list_test_acc, list_dev_precision, list_test_precision, list_dev_recall, list_test_recall, list_dev_f1score, list_test_f1score = main(n_epochs, model_name, model_save_flag, model_save_location, model_load_flag, model_load_location)
            
            overall_list_dev_acc.append(list_dev_acc) 
            overall_list_test_acc.append(list_test_acc) 
            overall_list_dev_precision.append(list_dev_precision)  
            overall_list_test_precision.append(list_test_precision) 
            overall_list_dev_recall.append(list_dev_recall)  
            overall_list_test_recall.append(list_test_recall)  
            overall_list_dev_f1score.append(list_dev_f1score)  
            overall_list_test_f1score.append(list_test_f1score) 

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Training epoch: 1
FINSIHED BATCH: 19 of 1115
FINSIHED BATCH: 39 of 1115
FINSIHED BATCH: 59 of 1115
FINSIHED BATCH: 79 of 1115
FINSIHED BATCH: 99 of 1115
FINSIHED BATCH: 119 of 1115
FINSIHED BATCH: 139 of 1115
FINSIHED BATCH: 159 of 1115
FINSIHED BATCH: 179 of 1115
FINSIHED BATCH: 199 of 1115
FINSIHED BATCH: 219 of 1115
FINSIHED BATCH: 239 of 1115
FINSIHED BATCH: 259 of 1115
FINSIHED BATCH: 279 of 1115
FINSIHED BATCH: 299 of 1115
FINSIHED BATCH: 319 of 1115
FINSIHED BATCH: 339 of 1115
FINSIHED BATCH: 359 of 1115
FINSIHED BATCH: 379 of 1115
FINSIHED BATCH: 399 of 1115
FINSIHED BATCH: 419 of 1115
FINSIHED BATCH: 439 of 1115
FINSIHED BATCH: 459 of 1115
FINSIHED BATCH: 479 of 1115
FINSIHED BATCH: 499 of 1115
FINSIHED BATCH: 519 of 1115
FINSIHED BATCH: 539 of 1115
FINSIHED BATCH: 559 of 1115
FINSIHED BATCH: 579 of 1115
FINSIHED BATCH: 599 of 1115
FINSIHED BATCH: 619 of 1115
FINSIHED BATCH: 639 of 1115
FINSIHED BATCH: 659 of 1115
FINSIHED BATCH: 679 of 1115
FINSIHED BATCH: 699 of 1115
FINSIHE

FINSIHED BATCH: 259 of 1115
FINSIHED BATCH: 279 of 1115
FINSIHED BATCH: 299 of 1115
FINSIHED BATCH: 319 of 1115
FINSIHED BATCH: 339 of 1115
FINSIHED BATCH: 359 of 1115
FINSIHED BATCH: 379 of 1115
FINSIHED BATCH: 399 of 1115
FINSIHED BATCH: 419 of 1115
FINSIHED BATCH: 439 of 1115
FINSIHED BATCH: 459 of 1115
FINSIHED BATCH: 479 of 1115
FINSIHED BATCH: 499 of 1115
FINSIHED BATCH: 519 of 1115
FINSIHED BATCH: 539 of 1115
FINSIHED BATCH: 559 of 1115
FINSIHED BATCH: 579 of 1115
FINSIHED BATCH: 599 of 1115
FINSIHED BATCH: 619 of 1115
FINSIHED BATCH: 639 of 1115
FINSIHED BATCH: 659 of 1115
FINSIHED BATCH: 679 of 1115
FINSIHED BATCH: 699 of 1115
FINSIHED BATCH: 719 of 1115
FINSIHED BATCH: 739 of 1115
FINSIHED BATCH: 759 of 1115
FINSIHED BATCH: 779 of 1115
FINSIHED BATCH: 799 of 1115
FINSIHED BATCH: 819 of 1115
FINSIHED BATCH: 839 of 1115
FINSIHED BATCH: 859 of 1115
FINSIHED BATCH: 939 of 1115
FINSIHED BATCH: 959 of 1115
FINSIHED BATCH: 979 of 1115
FINSIHED BATCH: 999 of 1115
FINSIHED BATCH: 1019

FINSIHED BATCH: 559 of 1115
FINSIHED BATCH: 579 of 1115
FINSIHED BATCH: 599 of 1115
FINSIHED BATCH: 619 of 1115
FINSIHED BATCH: 639 of 1115
FINSIHED BATCH: 659 of 1115
FINSIHED BATCH: 679 of 1115
FINSIHED BATCH: 699 of 1115
FINSIHED BATCH: 719 of 1115
FINSIHED BATCH: 739 of 1115
FINSIHED BATCH: 759 of 1115
FINSIHED BATCH: 779 of 1115
FINSIHED BATCH: 799 of 1115
FINSIHED BATCH: 819 of 1115
FINSIHED BATCH: 839 of 1115
FINSIHED BATCH: 859 of 1115
FINSIHED BATCH: 879 of 1115
FINSIHED BATCH: 899 of 1115
FINSIHED BATCH: 919 of 1115
FINSIHED BATCH: 939 of 1115
FINSIHED BATCH: 959 of 1115
FINSIHED BATCH: 979 of 1115
FINSIHED BATCH: 999 of 1115
FINSIHED BATCH: 1019 of 1115
FINSIHED BATCH: 1039 of 1115
FINSIHED BATCH: 1059 of 1115
FINSIHED BATCH: 1079 of 1115
FINSIHED BATCH: 1099 of 1115
Validation loss per 100 evaluation steps: 0.9805566072463989
Validation loss per 100 evaluation steps: 1.3963945384072785
DEV ACC: 0.6080373759305211
DEV Precision: 0.4646369324047216
DEV Recall: 0.4662458121253

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

...

FINSIHED BATCH: 499 of 1115
FINSIHED BATCH: 519 of 1115
FINSIHED BATCH: 539 of 1115
FINSIHED BATCH: 559 of 1115
FINSIHED BATCH: 579 of 1115
FINSIHED BATCH: 599 of 1115
FINSIHED BATCH: 619 of 1115
FINSIHED BATCH: 639 of 1115
FINSIHED BATCH: 659 of 1115
FINSIHED BATCH: 679 of 1115
FINSIHED BATCH: 699 of 1115
FINSIHED BATCH: 719 of 1115
FINSIHED BATCH: 739 of 1115
FINSIHED BATCH: 759 of 1115
FINSIHED BATCH: 779 of 1115
FINSIHED BATCH: 799 of 1115
FINSIHED BATCH: 819 of 1115
FINSIHED BATCH: 839 of 1115
FINSIHED BATCH: 859 of 1115
FINSIHED BATCH: 879 of 1115
FINSIHED BATCH: 899 of 1115
FINSIHED BATCH: 919 of 1115
FINSIHED BATCH: 939 of 1115
FINSIHED BATCH: 959 of 1115
FINSIHED BATCH: 979 of 1115
FINSIHED BATCH: 999 of 1115
FINSIHED BATCH: 1019 of 1115
FINSIHED BATCH: 1039 of 1115
FINSIHED BATCH: 1059 of 1115
FINSIHED BATCH: 1079 of 1115
FINSIHED BATCH: 1099 of 1115
Validation loss per 100 evaluation steps: 1.2441974878311157
Validation loss per 100 evaluation steps: 1.4670272776395967
DEV A

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Training epoch: 1
FINSIHED BATCH: 19 of 1115
FINSIHED BATCH: 39 of 1115
FINSIHED BATCH: 59 of 1115
FINSIHED BATCH: 79 of 1115
FINSIHED BATCH: 99 of 1115
FINSIHED BATCH: 119 of 1115
FINSIHED BATCH: 139 of 1115
FINSIHED BATCH: 159 of 1115
FINSIHED BATCH: 179 of 1115
FINSIHED BATCH: 199 of 1115
FINSIHED BATCH: 219 of 1115
FINSIHED BATCH: 239 of 1115
FINSIHED BATCH: 259 of 1115
FINSIHED BATCH: 279 of 1115
FINSIHED BATCH: 299 of 1115
FINSIHED BATCH: 319 of 1115
FINSIHED BATCH: 339 of 1115
FINSIHED BATCH: 359 of 1115
FINSIHED BATCH: 379 of 1115
FINSIHED BATCH: 399 of 1115
FINSIHED BATCH: 419 of 1115
FINSIHED BATCH: 439 of 1115
FINSIHED BATCH: 459 of 1115
FINSIHED BATCH: 479 of 1115
FINSIHED BATCH: 499 of 1115
FINSIHED BATCH: 519 of 1115
FINSIHED BATCH: 539 of 1115
FINSIHED BATCH: 559 of 1115
FINSIHED BATCH: 579 of 1115
FINSIHED BATCH: 599 of 1115
FINSIHED BATCH: 619 of 1115
FINSIHED BATCH: 639 of 1115
FINSIHED BATCH: 659 of 1115
FINSIHED BATCH: 679 of 1115
FINSIHED BATCH: 699 of 1115
FINSIHE

FINSIHED BATCH: 259 of 1115
FINSIHED BATCH: 279 of 1115
FINSIHED BATCH: 299 of 1115
FINSIHED BATCH: 319 of 1115
FINSIHED BATCH: 339 of 1115
FINSIHED BATCH: 359 of 1115
FINSIHED BATCH: 379 of 1115
FINSIHED BATCH: 399 of 1115
FINSIHED BATCH: 419 of 1115
FINSIHED BATCH: 439 of 1115
FINSIHED BATCH: 459 of 1115
FINSIHED BATCH: 479 of 1115
FINSIHED BATCH: 499 of 1115
FINSIHED BATCH: 519 of 1115
FINSIHED BATCH: 539 of 1115
FINSIHED BATCH: 559 of 1115
FINSIHED BATCH: 579 of 1115
FINSIHED BATCH: 599 of 1115
FINSIHED BATCH: 619 of 1115
FINSIHED BATCH: 639 of 1115
FINSIHED BATCH: 659 of 1115
FINSIHED BATCH: 679 of 1115
FINSIHED BATCH: 699 of 1115
FINSIHED BATCH: 719 of 1115
FINSIHED BATCH: 739 of 1115
FINSIHED BATCH: 759 of 1115
FINSIHED BATCH: 779 of 1115
FINSIHED BATCH: 799 of 1115
FINSIHED BATCH: 819 of 1115
FINSIHED BATCH: 839 of 1115
FINSIHED BATCH: 859 of 1115
FINSIHED BATCH: 879 of 1115
FINSIHED BATCH: 899 of 1115
FINSIHED BATCH: 919 of 1115
FINSIHED BATCH: 939 of 1115
FINSIHED BATCH: 959 

FINSIHED BATCH: 559 of 1115
FINSIHED BATCH: 579 of 1115
FINSIHED BATCH: 599 of 1115
FINSIHED BATCH: 619 of 1115
FINSIHED BATCH: 639 of 1115
FINSIHED BATCH: 659 of 1115
FINSIHED BATCH: 679 of 1115
FINSIHED BATCH: 699 of 1115
FINSIHED BATCH: 719 of 1115
FINSIHED BATCH: 739 of 1115
FINSIHED BATCH: 759 of 1115
FINSIHED BATCH: 779 of 1115
FINSIHED BATCH: 799 of 1115
FINSIHED BATCH: 819 of 1115
FINSIHED BATCH: 839 of 1115
FINSIHED BATCH: 859 of 1115
FINSIHED BATCH: 879 of 1115
FINSIHED BATCH: 899 of 1115
FINSIHED BATCH: 919 of 1115
FINSIHED BATCH: 939 of 1115
FINSIHED BATCH: 959 of 1115
FINSIHED BATCH: 979 of 1115
FINSIHED BATCH: 999 of 1115
FINSIHED BATCH: 1019 of 1115
FINSIHED BATCH: 1039 of 1115
FINSIHED BATCH: 1059 of 1115
FINSIHED BATCH: 1079 of 1115
FINSIHED BATCH: 1099 of 1115
Validation loss per 100 evaluation steps: 1.2617164850234985
FINSIHED BATCH: 39 of 1115
FINSIHED BATCH: 59 of 1115
FINSIHED BATCH: 79 of 1115
FINSIHED BATCH: 99 of 1115
FINSIHED BATCH: 119 of 1115
FINSIHED BATCH

In [6]:
print(best_dev_acc)

0.6501046836228288


In [7]:
print(best_epoch)

6


In [8]:
print(best_dev_precision)

0.5055399469202733


In [9]:
print(best_dev_recall)

0.47776187217963995


In [10]:
print(best_dev_f1score)

0.46452306566380386


In [11]:
print(overall_list_dev_acc)

[[0.6315330334987593, 0.6448317307692307, 0.6477977667493797, 0.6469060173697271, 0.6372130893300249, 0.6193199441687345, 0.6179823200992556, 0.6102085918114144, 0.6080373759305211, 0.6119533188585607], [0.6282374379652605, 0.6360111662531017, 0.6462275124069479, 0.6297107630272952, 0.6460724255583127, 0.6344990694789082, 0.6136980459057072, 0.6137755893300249, 0.6162182071960298, 0.6035010856079405], [0.6292067307692307, 0.6360111662531017, 0.6418850806451613, 0.6387058002481389, 0.6163539081885856, 0.6225379962779156, 0.6183700372208436, 0.6106738523573201, 0.6121471774193549, 0.6104799937965261], [0.6280048076923077, 0.6296526054590571, 0.6409933312655087, 0.6395587779156328, 0.6310290012406948, 0.6363213399503722, 0.6236429900744417, 0.6063508064516129, 0.6051876550868486, 0.6224992245657568], [0.6218207196029776, 0.6418075372208436, 0.6501046836228288, 0.6391129032258065, 0.6160631203473945, 0.6382793114143921, 0.6311647022332506, 0.6212779156327543, 0.6130195409429281, 0.61013104

In [12]:
print(overall_list_dev_precision)

[[0.45896844055465064, 0.49545969115229654, 0.5003205812841832, 0.5161720961627898, 0.4917461097561304, 0.4818015854019582, 0.47620695887073, 0.47022209175802687, 0.4646369324047216, 0.4666085658159921], [0.4518531964256932, 0.47109698693250474, 0.5032898204966788, 0.5068549143868198, 0.5015913572796111, 0.49197402779990645, 0.48017782484625854, 0.46922883067084187, 0.4735746209792693, 0.47283693308867913], [0.4691246801987946, 0.48728148178277125, 0.5120641591702036, 0.4924734159071326, 0.47050638029664293, 0.48635610037367677, 0.4811140825333333, 0.4723394962130976, 0.47910050191777465, 0.47085830846262766], [0.4464798467229925, 0.4671943549351645, 0.502376318100803, 0.5004359114668829, 0.4820498767953276, 0.48784767165420867, 0.47817215991549, 0.48028360106146784, 0.4640630311519054, 0.4705655175762838], [0.4636459689846553, 0.4914793272142943, 0.493033064988018, 0.5025758988051031, 0.5055399469202733, 0.4936796090568911, 0.5025015196070044, 0.4807355165697881, 0.4672180905970731, 0

In [13]:
print(overall_list_dev_recall)

[[0.3788258654312637, 0.4200720530670063, 0.48724175509481843, 0.4764099849511036, 0.46411609380474905, 0.47061618523785775, 0.4595612097874933, 0.446828982596484, 0.46624581212532784, 0.4436563208904983], [0.38073419421160215, 0.4724764247428396, 0.4585644029670635, 0.46595393605244856, 0.461471639986216, 0.46074157984645703, 0.4680637908824324, 0.4449433688616368, 0.4540731346534348, 0.484025146196788], [0.415156030830688, 0.42855515309660447, 0.42961232631619006, 0.4436100343896947, 0.4701681088744198, 0.4680753538726889, 0.4566689298018111, 0.45796267064922647, 0.46325178373643705, 0.45791531620015935], [0.38737014227756705, 0.46803031992951655, 0.47720515053586393, 0.4661426614397265, 0.4514390568243304, 0.45665653529630085, 0.44967896085990944, 0.46040347523470665, 0.44641157590432556, 0.4308847820609618], [0.4227232834954035, 0.4491202975619529, 0.4498749856491035, 0.47175828237649714, 0.47724239787140904, 0.462156564233241, 0.47776187217963995, 0.4521903190894976, 0.46341304481

In [14]:
print(overall_list_dev_f1score)

[[0.37233771853452924, 0.4188899694980533, 0.4647147994981794, 0.46221641145334114, 0.4503721064097314, 0.44986690234716736, 0.4410967126602655, 0.43032997606663614, 0.44529744388364395, 0.4298802313628807], [0.3779747822301601, 0.4457690292218414, 0.45472196016741, 0.4570539048313191, 0.45038706053296945, 0.45009598797466344, 0.44874888148766034, 0.4331517891389988, 0.4399726432492539, 0.4528735675326381], [0.4066304493660208, 0.4236220308058946, 0.4303129561221399, 0.43749940767494727, 0.446334478559108, 0.45422405756558637, 0.44074819423908657, 0.44458929605435443, 0.44536921114421396, 0.43707077025860863], [0.37719528667694685, 0.44404938924860077, 0.46376843518087124, 0.4578211047203233, 0.4436704730933077, 0.4425802140918271, 0.43685433229000975, 0.44332769894168667, 0.42757365239983075, 0.42208948670133833], [0.41467626294414367, 0.44425409431580276, 0.4398700284950027, 0.4578757377666152, 0.4616488422523307, 0.4502506758603767, 0.46452306566380386, 0.44206118818819096, 0.443127

In [15]:
#The best model is 3